In [1]:
import numpy as np
import pandas as pd
import pickle

import os

from data_load import load_data, load_meshes_coor_tria
from tv_utils import get_data_ij


from scipy.spatial.distance import squareform

from sklearn.model_selection import ShuffleSplit
from sklearn.cross_decomposition import PLSRegression

from sklearn.datasets import make_regression
path = '/cobrain/groups/ml_group/data/HCP/HCP/'
connec , thinkness, log_jac, unique_labels, labels, mean_area_eq, idx_subj_connec, idx_subj_think, idx_subj_logjac = load_data(path)

path_tria= "/cobrain/groups/ml_group/data/HCP/HCP/"
path_new_tria = '/cobrain/groups/ml_group/data/HCP/HCP/'

name_simp = '_200_mean_IC5.m'
# name_orig = '_200_mean.m' 

# orig_coord, orig_tria = load_meshes_coor_tria(path_tria, name_orig)
new_coord , new_tria = load_meshes_coor_tria(path_new_tria, name_simp)



path_res = '/home/ayagoz/connec/results/'
                   
Y = connec[:,0,:,:]
Y = np.array([squareform(one) for one in Y])
print(Y.shape)

m = int(len(new_coord)/2)
new_log_jac = np.concatenate([log_jac[:,0, :m], log_jac[:,1,:m]], axis = 1)

new_thick = np.concatenate([thinkness[:,0, :m], thinkness[:,1,:m]], axis = 1)

new_area = np.concatenate([mean_area_eq[0, :m] * np.exp(new_log_jac[:,:m]), 
                           mean_area_eq[1, :m] * np.exp(new_log_jac[:,m:])], axis = 1)

new_labels = np.concatenate([labels[0,:m], labels[1,:m]], axis = 0)

print('new ', new_log_jac.shape, new_thick.shape, new_area.shape, new_labels.shape)

X = np.concatenate([new_thick, new_log_jac, new_area], axis = 1)
X.shape, Y.shape



row = 0
results = pd.DataFrame(columns=['random_state', 'mean_test', 'mean_train'])

# components= [2, 5,]
# max_iter = [2000, 3000]


n = 50
m = 1000
R2_test = []
x_load = []
y_load = []
random_states = np.random.randint(0, 10000, 100)
with opne('rnd_states100', 'wb'):
    pickle.dump(random_states, f)
st = time.time()
for i in random_states:
    r2_train = []
    r2_test = []
    x_loc = []
    y_loc = []
    cv = ShuffleSplit(n_splits=2, test_size=0.2, random_state=i)
    for train_index, test_index in cv.split(X):
        X_train, y_train = X[train_index], Y[train_index]
        X_test, y_test = X[test_index], Y[test_index]
        plsr = PLSRegression(n_components=n, max_iter=m)
        plsr.fit(X_train, y_train)
        x_loc += [plsr.x_loadings_]
        y_loc += [plsr.y_loadings_]
        r2_train += [plsr.score(X_train, y_train)]
        r2_test += [plsr.score(X_test, y_test)]
        print('R2 train: {}, test: {}'.format(r2_train[-1], r2_test[-1]))
    print('MEAN R2 TRAIN: {}, TEST: {}'.format(np.mean(r2_train), np.mean(r2_test)))
    R2_test += [r2_test]
    x_load += [x_loc]
    y_load += [y_loc]
    results.loc[row] = [i, np.mean(r2_test), np.mean(r2_train)]
    row += 1
end = time.time() - st
print(end)

R2_test = np.array(R2_test)
x_load = np.array(x_load)
y_load = np.array(y_load)

R2_test.shape, x_load.shape, y_load.shape

R2_score = pd.DataFrame(columns=['random_state', 'r2_fold1', 'r2_fold2'])

for i in range(len(radsom_states)):
    R2_score.loc[i] = [random_states[i], R2_test[i][0], R2_test[i][1]]

R2_score.to_csv('R2_test_100rand_state.csv')

x_components = pd.DataFrame(columns=['random_state', 'comp_fold1', 'comp_fold2'])

for i in range(len(radsom_states)):
    x_components.loc[i] = [random_states[i], x_load[i][0], x_load[i][1]]

y_components = pd.DataFrame(columns=['random_state', 'comp_fold1', 'comp_fold2'])

for i in range(len(radsom_states)):
    y_components.loc[i] = [random_states[i], y_load[i][0], y_load[i][1]]

x_components.to_csv('x_components50_iter1000_rndst100')
y_components.to_csv('y_components50_iter1000_rndst100')

Connectomes:  (789, 2, 68, 68)
Thinkness:  (789, 2, 163842)
Log Jacob:  (789, 2, 163842)
Unique Labels:  (789, 2, 163842)
Mean Labels:  (2, 163842)
Mean Area eq:  (2, 163842)
Meshes coordinates shape:  (20484, 3)
Number of triangles of meshes:  (40960, 3)
(789, 2278)
new  (789, 20484) (789, 20484) (789, 20484) (20484,)


((789, 61452), (789, 2278))

In [9]:
import numpy as np
import pandas as pd
import pickle

import os
import time
from data_load import load_data, load_meshes_coor_tria
from tv_utils import get_data_ij


from scipy.spatial.distance import squareform

from sklearn.model_selection import ShuffleSplit
from sklearn.cross_decomposition import PLSRegression

from sklearn.datasets import make_regression
path = '/cobrain/groups/ml_group/data/HCP/HCP/'
connec , thinkness, log_jac, unique_labels, labels, mean_area_eq, idx_subj_connec, idx_subj_think, idx_subj_logjac = load_data(path)

path_tria= "/cobrain/groups/ml_group/data/HCP/HCP/"
path_new_tria = '/cobrain/groups/ml_group/data/HCP/HCP/'

name_simp = '_200_mean_IC5.m'
# name_orig = '_200_mean.m' 

# orig_coord, orig_tria = load_meshes_coor_tria(path_tria, name_orig)
new_coord , new_tria = load_meshes_coor_tria(path_new_tria, name_simp)



Y = connec[:,0,:,:]
Y = np.array([squareform(one) for one in Y])
print(Y.shape)

m = int(len(new_coord)/2)
new_log_jac = np.concatenate([log_jac[:,0, :m], log_jac[:,1,:m]], axis = 1)

new_thick = np.concatenate([thinkness[:,0, :m], thinkness[:,1,:m]], axis = 1)

new_area = np.concatenate([mean_area_eq[0, :m] * np.exp(new_log_jac[:,:m]), 
                           mean_area_eq[1, :m] * np.exp(new_log_jac[:,m:])], axis = 1)

new_labels = np.concatenate([labels[0,:m], labels[1,:m]], axis = 0)

print('new ', new_log_jac.shape, new_thick.shape, new_area.shape, new_labels.shape)

X = np.concatenate([new_thick, new_log_jac, new_area], axis = 1)
X.shape, Y.shape



row = 0
results = pd.DataFrame(columns=['random_state', 'mean_test', 'mean_train'])

# components= [2, 5,]
# max_iter = [2000, 3000]


n = 50
m = 1000
R2_test = []
x_load = []
y_load = []
# random_states = np.random.randint(0, 10000, 1000)
# with open('rnd_states100', 'wb') as f:
#     pickle.dump(random_states, f)
# st = time.time()
# for i in random_states:
#     r2_train = []
#     r2_test = []
#     x_loc = []
#     y_loc = []
#     cv = ShuffleSplit(n_splits=2, test_size=0.2, random_state=i)
#     for train_index, test_index in cv.split(X):
#         X_train, y_train = X[train_index], Y[train_index]
#         X_test, y_test = X[test_index], Y[test_index]
#         plsr = PLSRegression(n_components=n, max_iter=m)
#         plsr.fit(X_train, y_train)
#         x_loc += [plsr.x_loadings_]
#         y_loc += [plsr.y_loadings_]
#         r2_train += [plsr.score(X_train, y_train)]
#         r2_test += [plsr.score(X_test, y_test)]
#         print('R2 train: {}, test: {}'.format(r2_train[-1], r2_test[-1]))
#     print('MEAN R2 TRAIN: {}, TEST: {}'.format(np.mean(r2_train), np.mean(r2_test)))
#     R2_test += [r2_test]
#     x_load += [x_loc]
#     y_load += [y_loc]
#     results.loc[row] = [i, np.mean(r2_test), np.mean(r2_train)]
#     row += 1
# end = time.time() - st
# print(end)

# R2_test = np.array(R2_test)
# x_load = np.array(x_load)
# y_load = np.array(y_load)

# R2_test.shape, x_load.shape, y_load.shape

# R2_score = pd.DataFrame(columns=['random_state', 'r2_fold1', 'r2_fold2'])

# for i in range(len(radsom_states)):
#     R2_score.loc[i] = [random_states[i], R2_test[i][0], R2_test[i][1]]

# R2_score.to_csv('R2_test_100rand_state.csv')

# x_components = pd.DataFrame(columns=['random_state', 'comp_fold1', 'comp_fold2'])

# for i in range(len(radsom_states)):
#     x_components.loc[i] = [random_states[i], x_load[i][0], x_load[i][1]]

# y_components = pd.DataFrame(columns=['random_state', 'comp_fold1', 'comp_fold2'])

# for i in range(len(radsom_states)):
#     y_components.loc[i] = [random_states[i], y_load[i][0], y_load[i][1]]

# x_components.to_csv('x_components50_iter1000_rndst100')
# y_components.to_csv('y_components50_iter1000_rndst100')

Connectomes:  (789, 2, 68, 68)
Thinkness:  (789, 2, 163842)
Log Jacob:  (789, 2, 163842)
Unique Labels:  (789, 2, 163842)
Mean Labels:  (2, 163842)
Mean Area eq:  (2, 163842)
Meshes coordinates shape:  (20484, 3)
Number of triangles of meshes:  (40960, 3)
(789, 2278)
new  (789, 20484) (789, 20484) (789, 20484) (20484,)


In [10]:
from scipy.stats import chisquare,ttest_1samp
from scipy.special import ndtr

In [59]:
R2_test = np.random.normal(loc=0.1, size=(5,10))
chisquare(R2_test.reshape(-1), f_exp=0.1)

Power_divergenceResult(statistic=398.14816859732093, pvalue=3.3416354010128171e-56)

In [35]:
ttest_1samp(R2_test.reshape(-1), R2_test.mean()-0.5)

Ttest_1sampResult(statistic=3.3023865686763441, pvalue=0.0017948020109195553)

In [25]:
from scipy.spatial.distance import cosine

In [48]:
a = np.random.normal(size = 10)
b = np.random.normal(loc = 1,scale = 2,size = (10))
a.shape, b.shape

((10,), (10,))

In [49]:
cosine(a,b)

0.85285541873378956

In [ ]:
cosine()

In [60]:
print('ttest H_0=0. {}, H_0=0.05 {}, H_0=0.07 {}, H_0=mean {}'.format(ttest_1samp(R2_test.reshape(-1), 0.)[1],
                                                            ttest_1samp(R2_test.reshape(-1), 0.05)[1],
                                                            ttest_1samp(R2_test.reshape(-1), 0.07)[1],
                                                            ttest_1samp(R2_test.reshape(-1), R2_test.mean())[1],))

ttest H_0=0. 0.17968793243927889, H_0=0.05 0.3379812143689227, H_0=0.07 0.421734308202635, H_0=mean 1.0


In [61]:
t1=ttest_1samp(R2_test.reshape(-1), 0.)[1]
t2=ttest_1samp(R2_test.reshape(-1), 0.05)[1]
t3=ttest_1samp(R2_test.reshape(-1), 0.07)[1]
t4=ttest_1samp(R2_test.reshape(-1), R2_test.mean())[1]

In [68]:
x_load = np.random.normal(size = (10,2,2000,10))
y_load = np.random.normal(size = (10,2,2000,10))

In [74]:
len_idx_x_load = []
len_idx_y_load = []

cosine_x_load = []
cosine_y_load = []

print('for x_loadings')

for a,b in x_load:
    idx_all = set(np.arange(X.shape[-1]))
    for i in range(3):
        p1 = np.percentile(abs(a[:,i]), 90)
        p2 = np.percentile(abs(b[:,i]), 90)
        print('top 10 for a from {}, for b from {}'.format(p1, p2))
        idx1 = set(np.where(abs(a[:,i]) > p1)[0])
        idx2 = set(np.where(abs(b[:,i]) > p2)[0])
        idx = idx1.intersection(idx2)
        idx_all = idx_all.intersection(idx)
        len_idx_x_load += [len(idx_all)]
        print('len intersection a and b {} :'.format(len(idx_all)))
        diff = np.linalg.norm(a[:,i] - b[:,i])
        print('euclidian norm b/w components {}'.format(diff))
        cos = 1 - cosine(a[:,i], b[:,i])
        print('cosine b/w a & b {}'.format(cos))
        cosine_x_load += [cos]
        

print('\n\n\nfor y_loadings')        
idx_all = set(np.arange(X.shape[-1]))
for a,b in y_load:
    idx_all = set(np.arange(X.shape[-1]))
    for i in range(3):
        p1 = np.percentile(abs(a[:,i]), 90)
        p2 = np.percentile(abs(b[:,i]), 90)
        print('top 10 for a from {}, for b from {}'.format(p1, p2))
        idx1 = set(np.where(abs(a[:,i]) > p1)[0])
        idx2 = set(np.where(abs(b[:,i]) > p2)[0])
        idx = idx1.intersection(idx2)
        idx_all = idx_all.intersection(idx)
        len_idx_y_load += [len(idx_all)]
        print('len intersection a and b {} :'.format(len(idx_all)))
        cos = 1 - cosine(a[:,i], b[:,i])
        diff = np.linalg.norm(a[:,i] - b[:,i])
        print('euclidian norm b/w components {}'.format(diff))
        print('cosine b/w a & b {}'.format(cos))
        cosine_y_load += [cos]

for x_loadings
top 10 for a from 1.6040898125373393, for b from 1.6481763180923947
len intersection a and b 17 :
euclidian norm b/w components 64.7864447917465
cosine b/w a & b -0.04297054292346347
top 10 for a from 1.6312120940437085, for b from 1.6436613683170784
len intersection a and b 0 :
euclidian norm b/w components 64.54792499073284
cosine b/w a & b -0.021997662002289475
top 10 for a from 1.6212018389875003, for b from 1.670628417403809
len intersection a and b 0 :
euclidian norm b/w components 61.86328053533302
cosine b/w a & b 0.018382884425825807
top 10 for a from 1.6601135515248346, for b from 1.6236233426265367
len intersection a and b 21 :
euclidian norm b/w components 63.200562619461905
cosine b/w a & b -0.00921212453155773
top 10 for a from 1.654788171201149, for b from 1.6071376039356418
len intersection a and b 0 :
euclidian norm b/w components 62.5518565655545
cosine b/w a & b -0.006161553367585615
top 10 for a from 1.5696445779614547, for b from 1.6848902318120385
l

top 10 for a from 1.6633280424989574, for b from 1.6529666518999722
len intersection a and b 26 :
euclidian norm b/w components 62.47944329996124
cosine b/w a & b 0.02381757534158191
top 10 for a from 1.6619851030768906, for b from 1.6636175049231774
len intersection a and b 0 :
euclidian norm b/w components 63.95170712975395
cosine b/w a & b -0.022992299030731855
top 10 for a from 1.5902193013396804, for b from 1.6277171732711755
len intersection a and b 0 :
euclidian norm b/w components 61.96183086107266
cosine b/w a & b -0.002302862024446517
top 10 for a from 1.6423747719524346, for b from 1.6812782053684527
len intersection a and b 25 :
euclidian norm b/w components 63.001026176201044
cosine b/w a & b 0.0033372561715807603
top 10 for a from 1.5776331542552628, for b from 1.6952601558417495
len intersection a and b 0 :
euclidian norm b/w components 62.720937087986556
cosine b/w a & b 0.0011264448114480796
top 10 for a from 1.6299864154756132, for b from 1.652399887632138
len interse

In [75]:
print('mean cosine of x_load {}, y_load {}'.format(np.mean(cosine_x_load), np.mean(cosine_y_load)))

mean cosine of x_load -0.001954200968719877, y_load 0.0009425062488791602


In [6]:
import numpy as np
import pandas as pd
import pickle

import os

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from data_load import load_data, load_meshes_coor_tria
from tv_utils import get_data_ij


from scipy.spatial.distance import squareform

from sklearn.model_selection import ShuffleSplit
from sklearn.cross_decomposition import PLSRegression


path = '/cobrain/groups/ml_group/data/HCP/HCP/'
connec , thinkness, log_jac, unique_labels, labels, mean_area_eq, idx_subj_connec, idx_subj_think, idx_subj_logjac = load_data(path)

# path_tria= "/cobrain/groups/ml_group/data/HCP/HCP/"
path_new_tria = '/home/bgutman/datasets/HCP/Simplified_templates/'

name_simp = '_200_mean_IC5.m'
# name_orig = '_200_mean.m' 

# orig_coord, orig_tria = load_meshes_coor_tria(path_tria, name_orig)
new_coord , new_tria = load_meshes_coor_tria(path_new_tria, name_simp)


cv = ShuffleSplit(n_splits = 3, test_size=0.2)


path_res = '/home/ayagoz/connec/results/'
                   
Y = connec[:,0,:,:]
Y = np.array([squareform(one) for one in Y])
print(Y.shape)

m = int(len(new_coord)/2)
new_log_jac = np.concatenate([log_jac[:,0, :m], log_jac[:,1,:m]], axis = 1)

new_thick = np.concatenate([thinkness[:,0, :m], thinkness[:,1,:m]], axis = 1)

new_area = np.concatenate([mean_area_eq[0, :m] * np.exp(new_log_jac[:,:m]), 
                           mean_area_eq[1, :m] * np.exp(new_log_jac[:,m:])], axis = 1)

new_labels = np.concatenate([labels[0,:m], labels[1,:m]], axis = 0)

print('new ', new_log_jac.shape, new_thick.shape, new_area.shape, new_labels.shape)

X = np.concatenate([new_thick, new_log_jac, new_area], axis = 1)
X.shape, Y.shape

row = 0
results = pd.DataFrame(columns=['n_components', 'max_iter',  'mean_test', 'mean_train'])

components= [2, 5, 10, 50, 80, 99, 130]
max_iter = [500, 1000, 2000]

for n in components:
    for m in max_iter:
        r2_train = []
        r2_test = []
        plsr = PLSRegression(n_components=n, max_iter=m)
        print('PARAMS n_components: {}, max_iter: {}'.format(n, m))
        for train_index, test_index in cv.split(X):
            X_train, y_train = X[train_index], Y[train_index]
            X_test, y_test = X[test_index], Y[test_index]
            plsr.fit(X_train, y_train)
            r2_train += [plsr.score(X_train, y_train)]
            r2_test += [plsr.score(X_test, y_test)]
            print('R2 train: {}, test: {}'.format(r2_train[-1], r2_test[-1]))
        print('MEAN R2 TRAIN: {}, TEST: {}'.format(np.mean(r2_train), np.mean(r2_test)))
    results.loc[row] = [n, m, np.mean(r2_test), np.mean(r2_train)]
    row += 1



results.to_csv('/home/ayagoz/connec/results/PLSReg/all_ICO5_to_edge')        

Connectomes:  (789, 2, 68, 68)
Thinkness:  (789, 2, 163842)
Log Jacob:  (789, 2, 163842)
Unique Labels:  (789, 2, 163842)
Mean Labels:  (2, 163842)
Mean Area eq:  (2, 163842)
Meshes coordinates shape:  (20484, 3)
Number of triangles of meshes:  (40960, 3)
(789, 2278)


In [76]:
import numpy as np
import pandas as pd
import pickle

import os

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from data_load import load_data, load_meshes_coor_tria
from tv_utils import get_data_ij


from scipy.spatial.distance import squareform

from sklearn.model_selection import ShuffleSplit
from sklearn.cross_decomposition import PLSRegression


path = '/cobrain/groups/ml_group/data/HCP/HCP/'
connec , thinkness, log_jac, unique_labels, labels, mean_area_eq, idx_subj_connec, idx_subj_think, idx_subj_logjac = load_data(path)

# path_tria= "/cobrain/groups/ml_group/data/HCP/HCP/"
path_new_tria = '/cobrain/groups/ml_group/data/HCP/HCP/'

name_simp = '_200_mean_IC5.m'
# name_orig = '_200_mean.m' 

# orig_coord, orig_tria = load_meshes_coor_tria(path_tria, name_orig)
new_coord , new_tria = load_meshes_coor_tria(path_new_tria, name_simp)


cv = ShuffleSplit(n_splits = 3, test_size=0.2)


path_res = '/home/ayagoz/connec/results/'
                   
Y = connec[:,0,:,:]
Y = np.array([squareform(one) for one in Y])
print(Y.shape)

m = int(len(new_coord)/2)
new_log_jac = np.concatenate([log_jac[:,0, :m], log_jac[:,1,:m]], axis = 1)

new_thick = np.concatenate([thinkness[:,0, :m], thinkness[:,1,:m]], axis = 1)

new_area = np.concatenate([mean_area_eq[0, :m] * np.exp(new_log_jac[:,:m]), 
                           mean_area_eq[1, :m] * np.exp(new_log_jac[:,m:])], axis = 1)
new_vol = np.concatenate([mean_area_eq[0, :m] * np.exp(new_log_jac[:,:m]) * new_thick[:,:m], 
                           mean_area_eq[1, :m] * np.exp(new_log_jac[:,m:])*new_thick[:,:m]], axis = 1)

new_labels = np.concatenate([labels[0,:m], labels[1,:m]], axis = 0)

print('new ', new_log_jac.shape, new_thick.shape, new_area.shape, new_labels.shape)

X = np.concatenate([new_thick, new_log_jac, new_area, new_vol], axis = 1)
X.shape, Y.shape

row = 0
results = pd.DataFrame(columns=['n_components', 'max_iter',  'mean_test', 'mean_train'])

components= [2, 5, 10, 50, 80, 99, 130]
max_iter = [500, 1000, 2000]

for n in components:
    for m in max_iter:
        r2_train = []
        r2_test = []
        plsr = PLSRegression(n_components=n, max_iter=m)
        print('PARAMS n_components: {}, max_iter: {}'.format(n, m))
        for train_index, test_index in cv.split(X):
            X_train, y_train = X[train_index], Y[train_index]
            X_test, y_test = X[test_index], Y[test_index]
            plsr.fit(X_train, y_train)
            r2_train += [plsr.score(X_train, y_train)]
            r2_test += [plsr.score(X_test, y_test)]
            print('R2 train: {}, test: {}'.format(r2_train[-1], r2_test[-1]))
        print('MEAN R2 TRAIN: {}, TEST: {}'.format(np.mean(r2_train), np.mean(r2_test)))
    results.loc[row] = [n, m, np.mean(r2_test), np.mean(r2_train)]
    row += 1



results.to_csv('/home/ayagoz/connec/results/PLSReg/all_ICO5_vol_to_edge')

Connectomes:  (789, 2, 68, 68)
Thinkness:  (789, 2, 163842)
Log Jacob:  (789, 2, 163842)
Unique Labels:  (789, 2, 163842)
Mean Labels:  (2, 163842)
Mean Area eq:  (2, 163842)
Meshes coordinates shape:  (20484, 3)
Number of triangles of meshes:  (40960, 3)
(789, 2278)
new  (789, 20484) (789, 20484) (789, 20484) (20484,)


In [77]:
X.shape

(789, 81936)